In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path


In [2]:


DATA_RECOMENDATIONS = Path("data/recommendations")
DATA_SUGGESTIONS = Path("data/suggestions")

for p in (DATA_RECOMENDATIONS, DATA_SUGGESTIONS):
    p.mkdir(parents=True, exist_ok=True)

print(f"Data directories ensured: {DATA_RECOMENDATIONS}, {DATA_SUGGESTIONS}")

Data directories ensured: data\recommendations, data\suggestions


In [3]:
def latest_file_in_directory(directory=DATA_RECOMENDATIONS):
    latest_file = max(
        f for f in os.listdir(directory)
        if f.startswith("trade_suggestions_") and f.endswith(".csv")
    )
    return latest_file

In [4]:
latest_file = latest_file_in_directory(DATA_RECOMENDATIONS)
df_all = pd.read_csv(os.path.join(DATA_RECOMENDATIONS, latest_file))
df_all.shape

(10, 16)

In [5]:
df_all = df_all.rename(columns={"Unnamed: 0": "ticker"})
df_all.head()

,ticker,price,ma50,avg_HotScore,median_HotScore,max_HotScore,avg_TrendScore,avg_MomentumScore,avg_VolumeScore,avg_VolatilityScore,avg_VolumeSpike,avg_regularMarketChangePercent,avg_regularMarketPrice,appearances,avg_HotScore_persistent,rule_score
0,IPGP,153.535004,86.803100,0.984375,0.984375,0.984375,0.971875,0.993750,0.975000,0.990625,6.839070,35.482414,150.25,1,0.984375,4
1,EQIX,962.354980,790.330701,0.975313,0.975313,0.975313,0.993750,0.975000,0.956250,1.000000,4.742605,10.414742,957.87,1,0.975313,4
2,TPH,46.345001,33.959100,0.963435,0.963435,0.964535,0.809172,1.000000,0.992541,0.925643,7.623340,26.888642,46.40,1,0.963435,4
3,CGNX,57.580002,39.443200,0.960625,0.960625,0.960625,0.806250,0.996875,0.978125,0.943750,7.075029,36.346733,58.67,1,0.960625,4
4,CROX,93.489998,86.943000,0.950000,0.950000,0.950000,0.890625,0.984375,0.934375,0.946875,3.698684,19.013653,98.46,1,0.950000,4


In [17]:
agg_map = {
    "price": "mean",
    "ma50": "mean",
    "appearances": "sum",
    "avg_HotScore": "mean",
    "rule_score": "max"
}

df_outcome = (
    df_all
    .groupby("ticker", as_index=False)
    .agg(agg_map)
)

df_outcome.sort_values("avg_HotScore", ascending=False).head()

,ticker,price,ma50,appearances,avg_HotScore,rule_score
4,IPGP,153.535004,86.803100,1,0.984375,4
2,EQIX,962.354980,790.330701,1,0.975313,4
7,TPH,46.345001,33.959100,1,0.963435,4
0,CGNX,57.580002,39.443200,1,0.960625,4
1,CROX,93.489998,86.943000,1,0.950000,4


In [ ]:
today = datetime.now().strftime("%Y%m%d")
out_file = DATA_SUGGESTIONS / f"trade_suggestions_{today}.csv"
df_outcome.to_csv(out_file, index=False)